In [1]:
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveFNN
from ray.air import session
from ray import tune
data_splits_dict = get_data(truncate=False, with_features=True)[0]

In [ ]:
train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]
configuration = Configuration()
hp_search_space = {
    "num_hidden_dimensions": tune.grid_search([[5],[10], [20]]),
    "context_length":tune.grid_search([4, 52, 104]),
    "epochs":tune.grid_search([5, 10, 20]),
    "prediction_length" : tune.grid_search([4, 52, 104]), 
    "num_batches_per_epoch":tune.grid_search([25, 50, 100]),
    "batch_normalization":tune.grid_search([True, False]),
    "batch_size":tune.grid_search([20, 32, 64]),
    }

tuner = tune.Tuner(
    tune.with_parameters(objectiveFNN, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 5,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=9,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("FNN_Hyperparameter_results_19_05.csv")

100%|██████████| 25/25 [00:06<00:00,  4.16it/s, epoch=5/5, avg_epoch_loss=1.35]
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 41922it [00:02, 15502.72it/s]
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 41922it [00:04, 9170.13it/s]
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 41922it [00:03, 13833.23it/s]
Running evaluation: 41922it [00:03, 12988.37it/s]
Running evaluation: 41922it [00:03, 13827.96it/s]
Running evaluation: 41922it [00:02, 16043.52it/s]
Running evaluation: 41922it [00:03, 10716.51it/s]
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 41922it [00:07, 5518.31it/s]
Running evaluation: 41922it [00:07, 5880.14it/s]
(objectiveFNN pid=2281216) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/d

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,mean_WIS,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
objectiveFNN_cca3c_00000,2023-05-19_12-51-52,True,,e9763982d9f44958bafe723f00739766,"0_batch_normalization=True,batch_size=20,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,725.127,172.22.1.197,2281143,151.829,151.829,151.829,1684493512,0,,1,cca3c_00000,0.0067246
objectiveFNN_cca3c_00001,2023-05-19_12-51-17,True,,2411c25cb99243ef8981dbd10b4c8af2,"1_batch_normalization=False,batch_size=20,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,696.604,172.22.1.197,2281216,111.126,111.126,111.126,1684493477,0,,1,cca3c_00001,0.0054543
objectiveFNN_cca3c_00002,2023-05-19_12-52-14,True,,59d51232e7f444cda56bdcbd703c64d9,"2_batch_normalization=True,batch_size=32,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,768.77,172.22.1.197,2281218,167.897,167.897,167.897,1684493534,0,,1,cca3c_00002,0.00670362
objectiveFNN_cca3c_00003,2023-05-19_12-52-16,True,,19b81913659146108e6732ba1bc02d6d,"3_batch_normalization=False,batch_size=32,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,781.484,172.22.1.197,2281220,170.288,170.288,170.288,1684493536,0,,1,cca3c_00003,0.00742674
objectiveFNN_cca3c_00004,2023-05-19_12-52-13,True,,ff2f0582dd5241ddbd068f650611015b,"4_batch_normalization=True,batch_size=64,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,737.833,172.22.1.197,2281222,166.995,166.995,166.995,1684493533,0,,1,cca3c_00004,0.00670695
objectiveFNN_cca3c_00005,2023-05-19_12-51-36,True,,0e458f3f47b440bc84bbbf459f92e578,"5_batch_normalization=False,batch_size=64,context_length=4,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,807.711,172.22.1.197,2281224,130.379,130.379,130.379,1684493496,0,,1,cca3c_00005,0.0052259
objectiveFNN_cca3c_00006,2023-05-19_12-51-58,True,,3f5a0f0f0b1849329eded6c04b55ade2,"6_batch_normalization=True,batch_size=20,context_length=52,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,762.218,172.22.1.197,2281226,153.037,153.037,153.037,1684493518,0,,1,cca3c_00006,0.00481415
objectiveFNN_cca3c_00007,2023-05-19_12-52-16,True,,b1e02dd3478c4436ba30460137edb81a,"7_batch_normalization=False,batch_size=20,context_length=52,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,801.767,172.22.1.197,2281228,170.079,170.079,170.079,1684493536,0,,1,cca3c_00007,0.00560546
objectiveFNN_cca3c_00008,2023-05-19_12-52-09,True,,cdd245ccdbff402d94e9ccb50ef14e00,"8_batch_normalization=True,batch_size=32,context_length=52,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,705.562,172.22.1.197,2281230,162.997,162.997,162.997,1684493529,0,,1,cca3c_00008,0.00578475
objectiveFNN_cca3c_00009,2023-05-19_12-52-52,True,,2411c25cb99243ef8981dbd10b4c8af2,"9_batch_normalization=False,batch_size=32,context_length=52,epochs=5,num_batches_per_epoch=25,num_hidden_dimensions=5,prediction_length=4",econ-stat-rr01,1,772.911,172.22.1.197,2281216,94.0845,94.0845,94.0845,1684493572,0,,1,cca3c_00009,0.0054543


  0%|          | 0/25 [00:00<?, ?it/s]m 
(objectiveFNN pid=2281224) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=2281224)   return arr.astype(dtype, copy=True)
  0%|          | 0/25 [00:00<?, ?it/s]m 
(objectiveFNN pid=2281143) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=2281143)   return arr.astype(dtype, copy=True)
  0%|          | 0/25 [00:00<?, ?it/s]m 
(objectiveFNN pid=2281226) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=2281226)   return arr.astype(dtype, copy=True)
  0%|          | 0/25 [00:00<?, ?it/s]m 
(objectiveFNN pid=2281230) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pand